# VinBigData Chest X-ray Abnormalities 

## Dataset Kaggle
***
- [VinBigData Chest X-ray Abnormalities](https://www.kaggle.com/awsaf49/vinbigdata-original-image-dataset)

## Contexto do problema
***
> Distúrbios pulmonares são doenças que afeta muitas pessoas nos países em desenvolvimento. Embora o tratamento seja possível, ela requer um diagnóstico preciso primeiro. Nos projetos de países, há em muitos casos máquinas de raio-X disponíveis (por meio de projetos de baixo custo e doações), mas muitas vezes falta a experiência radiológica para avaliar com precisão as imagens. Um algoritmo que pudesse realizar essa tarefa de forma rápida e barata poderia melhorar drasticamente a capacidade de diagnosticar e, em última análise, tratar a doença.  
***
> Em países mais desenvolvidos, a radiografia de raio-X é frequentemente usada para rastrear recém-chegados e determinar a elegibilidade para uma autorização de trabalho. A tarefa de examinar manualmente as imagens é demorada e um algoritmo pode aumentar a eficiência, melhorar o desempenho e, por fim, reduzir o custo dessa triagem.   

## Dataset 3: VinBigData Original Image Dataset
***
O Vingroup Big Data Institute (VinBigdata) concentra-se especificamente no campo da Ciência de Dados. A VinBigdata foi fundada em agosto de 2018 como parte de um desenvolvimento estratégico chave para o objetivo do Vingroup de se tornar uma empresa de Tecnologia - Industriais - Serviços de classe mundial, com a tecnologia no centro do palco. O conjunto de dados coletados do VinBigdata, compreende 18.000 varreduras CXR póstero-anterior (PA) em formato DICOM (convertidas para .jpg), cujas identificações foram removidas para proteger a privacidade do paciente. Todas as imagens foram marcadas por um painel de radiologistas experientes quanto à presença de 14 achados radiográficos críticos.

## Referências
***
- Ha Q. Nguyen, Khanh Lam, Linh T. Le, Hieu H. Pham, Dat Q. Tran, Dung B. Nguyen, Dung D. Le, Chi M. Pham, Hang T. T. Tong, Diep H. Dinh, Cuong D. Do, Luu T. Doan, Cuong N. Nguyen, Binh T. Nguyen, Que V. Nguyen, Au D. Hoang, Hien N. Phan, Anh T. Nguyen, Phuong H. Ho, Dat T. Ngo, Nghia T. Nguyen, Nhan T. Nguyen, Minh Dao, & Van Vu. (2021). **VinDr-CXR: An open dataset of chest X-rays with radiologist's annotations**.


## Importação dos pacotes

In [28]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

## Indexando todas as imagens disponíveis no dataset
***
- `pandas`: ferramenta de análise e manipulação de dados.

In [8]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('../datasets/archive (1)/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '../datasets/archive (1)/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [103]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 13948


## Extraindo os rótulos das imagens
***

#### Distúrbios datados
- `No finding [14]`: a observação data o não achado de nenhuma das doenças abaixo.
- `Aortic enlargement [0]`: protuberâncias anormais na aorta.
- `Cardiomegaly [3]`: coração aumentado.
- `Pleural thickening [11]`: tecido cicatricial desenvolvido no revestimento dos pulmões ou na pleura.
- `Pulmonary fibrosis [13]`: doença pulmonar que ocorre quando o tecido pulmonar fica danificado e com cicatrizes.
- `Nodule/Mass [8]`: lesão pulmonar bem definida, discreta, aproximadamente circular e com 3 cm ou menos de diâmetro.
- `Lung Opacity [7]`: um nódulo do pulmão pode ser visível em uma radiografia ou tomografia do tórax de rotina.
- `Pleural effusion [10]`: acúmulo de líquido entre os tecidos que revestem os pulmões e o tórax.
- `Other lesion [9]`: outros tipos de lesões detectadas nas radiografias.
- `Infiltration [6]`: distúrbio que afeta tanto o pulmão quanto o coração.
- `ILD [5]`: grupo de doenças que causam a formação progressiva de cicatrizes no tecido pulmonar.
- `Calcification [2]`: processo no qual os sais de cálcio se acumulam nos tecidos moles, tornando-os mais rígidos.
- `Consolidation [4]`: refere-se aos espaços aéreos alveolares sendo preenchidos com fluido.
- `Atelectasis [1]`: colapso completo ou parcial de um pulmão ou de uma seção (lóbulo) de um pulmão.
- `Pneumothorax [12]`: ocorre quando o ar escoa para o espaço entre os pulmões e a parede torácica.

### Rótulos removidos
- `class_name[Aortic enlargement, Cardiomegaly, Other lesion, Consolidation]` == `class_id[0, 3, 9, 4]`

#### Rótulos esperados
- `[0]` caso normal;
- `[1]` caso anormal;

In [41]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [84]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']
dataframe = dataframe[dataframe.class_name != 'Consolidation']

In [86]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

In [104]:
# removendo as imagens repetidas
normal_path = list(set(normal_cases['image_path']))
abnormal_path = list(set(abnormal_cases['image_path']))

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data = pd.DataFrame(normal_path, columns = ['filepath'])
normal_data['target'] = 0
abnormal_data = pd.DataFrame(abnormal_path, columns = ['filepath'])
abnormal_data['target'] = 1

In [114]:
print('quantidade de dados rotulados como normais:', len(normal_path))
print('quantidade de dados rotulados como anormais:', len(abnormal_path))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3342


In [107]:
# concatenando os dataframes de casos normais e anormais
full_data = pd.concat([normal_data, abnormal_data])
# criando um dataframe para os rótulos, removendo-os do dataframe que contém as imagens
full_labels = pd.DataFrame(full_data.pop('target'), columns = ['target'])

## Dados [Treinamento - Validação - Teste]
***
- `scikit-learn:` biblioteca de aprendizado de máquina de código aberto.

In [110]:
# separando os dados de treinamento e de teste
train_X, test_X, train_y, test_y = train_test_split(full_data, full_labels, stratify = full_labels,
                                                    test_size = 0.2, random_state = 42)

In [111]:
# adicionando os rótulos no dataframe dos dados de treinamento
train_X['target'] = train_y['target']

# carregando os rótulos do conjunto de treinamento e removendo-os do dataframe
labels = train_X.pop('target')

In [112]:
# separando os dados de validação dos dados de treinamento
train_X, validation_X, train_y, validation_y = train_test_split(train_X, labels, stratify = labels,
                                                                test_size = 0.2, random_state = 42)

In [113]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_X))
print('quantidade de rótulos de treinamento:', len(train_y))
print('quantidade de imagens de teste:', len(test_X))
print('quantidade de rótulos de teste:', len(test_y))
print('quantidade de imagens de validação:', len(validation_X))
print('quantidade de rótulos de validação:', len(validation_y))

quantidade de imagens de treinamento: 8926
quantidade de rótulos de treinamento: 8926
quantidade de imagens de teste: 2790
quantidade de rótulos de teste: 2790
quantidade de imagens de validação: 2232
quantidade de rótulos de validação: 2232
